# Imports 

In [195]:
import json; import os; import pandas as pd; import spacy; import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

 # Functions
 Vader Setiment\
 Spacy 

In [ ]:
sentimentSet = []
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print("Overall sentiment dictionary is : ", sentiment_dict)
    sentimentSet.append(sentiment_dict)
    
nlp = spacy.load("en_core_web_sm")

# Jason file manipulation



In [ ]:
# Counting jason files in directory 
folders = os.listdir("edhrec Slack export Sep 4 2016 - Oct 23 2019"); folderSize = (len(folders)); folderNames = [0]*folderSize
ii = 0

### Ignoring json files in folder (channels.json, integration_ logs.json, users.json)
for count in range(len(folderNames)):

    folderNames[ii] = folders[ii]   
    ii += 1

    if folderNames[ii-1] == "channels.json":
        folderNames[ii-1] = 0

    
    if folderNames[ii-1] == "integration_logs.json":
        folderNames[ii-1] = 0
    
    if folderNames[ii-1] == "users.json":
        folderNames[ii-1] = 0
###

### Removing placed zeros             
try:
    while True:
        folderNames.remove(0)
except ValueError:
    pass 
###



# Jason files to Python dictionaries

In [ ]:
fileList = []; folderList = []; folderPath = [0]*len(folderNames); mainPath = []; slackContent = []; all_possible_keys = set(); channelsDict = {'Channel':[]}

jj = 0; ii = 0

### Conversion of json files to python dictionaries and collection of key value pairs
for Paths in folderNames:
    
    mainPath1 = 'C:/Users/solid/Documents/Scripts/Code/'
    mainPath2 = 'MindStand/edhrec Slack export Sep 4 2016 - Oct 23 2019/'
    folderPath[ii] = folderNames[ii]
    folderList.append(os.path.join(mainPath1,mainPath2,folderPath[ii]))
    fileIter = os.listdir(folderList[ii])
    
    for d in range(len(fileIter)):
        fileList.append(fileIter[d])  
        mainPath.append(os.path.join(folderList[ii],fileList[d]))
        mainPath[jj] = mainPath[d].replace('\\','/')
        with open(mainPath[ii], encoding="utf8") as f:
            data = json.load(f)
            slackContent.append(data)
            for d in data:
                channelsDict['Channel'].append(Paths)
                for k in d.keys():
                    all_possible_keys.add(k)

        jj += 1
    ii += 1
###

# Dataframe Creation

In [ ]:
### Creation of dataframe based on a key: [list of values] organization
main_data = {}
for stuff in slackContent:
    for p in stuff:
        for field in list(all_possible_keys):
            if field not in main_data:
                main_data[field] = []
            if field in p:
                main_data[field].append(p[field])
            else:
                main_data[field].append(None)
dataframe = pd.DataFrame.from_dict(main_data)
df = dataframe[['text','thread_ts','user']]
channelDataFrame = pd.DataFrame.from_dict(channelsDict)
###

# Spacy application

In [ ]:
### Iterating through dataframe and applying spacy
dataStart = 2000
dataEnd =  2100
dataSpan = range(0,len(df),1)
test_nlp = [0]*len(dataSpan)
ii = 0
for row in dataSpan:
    test_nlp[ii] = nlp(df.text[row])
    ii += 1 
###

# Creation of Noun, Verb, and Adjective sets

In [ ]:
wordDataList = []
# DataFrames = pd.DataFrame()
ii = 0
for row in test_nlp:
    sentence = [row.text]
    noun = [token.text for token in test_nlp[ii] if token.pos_ == "NOUN"]
    Verb = [token.text for token in test_nlp[ii] if token.pos_ == "VERB"]
    Adj = [token.text for token in test_nlp[ii] if token.pos_ == "ADJ"]
    pt1 = sentence
    pt2 = noun
    pt1_pt2 = sentence,noun
    wordDataList.append([pt1_pt2])
    iterDataFrame = pd.DataFrame(wordDataList[0], columns=['Sentence','Nouns'])
    # wordDataFrame = DataFrames.append(iterDataFrame)
    wordDataFrame = pd.DataFrame().append(iterDataFrame)
    
    ii+=1
# wordDataFrame = pd.DataFrame(wordDataList[-1],columns=['sentence','Noun'])
# wordDataFrame = pd.DataFrame({'sentence':pt1,'Noun':pt2})




# Spacy Application

In [ ]:
wordDataList = []
iterDataFrame = [0]*len(test_nlp)
# DataFrames = pd.DataFrame()
ii = 0
for row in test_nlp:
    Sentence = [row.text]
    Noun = [token.text for token in test_nlp[ii] if token.pos_ == "NOUN"]
    Verb = [token.text for token in test_nlp[ii] if token.pos_ == "VERB"]
    Adj = [token.text for token in test_nlp[ii] if token.pos_ == "ADJ"]
    pt1 = sentence
    pt2 = Noun
    pt3 = Verb
    pt4 = Adj
    pts = Sentence,Noun,Verb,Adj
    wordDataList.append(pts)
    ii+=1

DataFrame = pd.DataFrame(wordDataList,columns=['Sentence','Noun','Verb','Adjective'])
wordDataFrame = DataFrame.join(channelDataFrame).join(dataframe['user'])
wordDataFrame

##### List items as strings

In [ ]:
def clean_alt_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_

In [ ]:
meme= wordDataFrame
meme = meme.explode('Sentence')
meme['Sentence'] = meme['Sentence'].str.strip()



In [201]:
meme['Sentence'] = meme.loc[meme['Sentence'].str.contains(r'https')==False]
meme = meme.loc[meme['Sentence'].str.contains(r'@')==False]
meme = meme.loc[meme['Sentence'].str.contains(r'::')==False]
meme = meme.loc[meme['Sentence'].str.contains(r'\\')==False]
meme = meme.loc[meme['Sentence'].str.contains(r'\+')==False]
# meme = meme.dropna(subset = ["Sentence"],inplace=True)
# print(len(wordDataFrame))
# print(len(meme))
# for index, row in meme.iterrows():
#     if row.any() == 'NaN':
#         meme.drop(index,inplace=True)
# print(len(meme['Sentence']))
# print(len(meme['Noun']))
meme.dropna(inplace=True)
meme['Sentence'].replace('',np.nan,inplace=True)
meme.dropna(subset=['Sentence'],inplace=True)
print(len(wordDataFrame))
print(len(meme))
meme.drop_duplicates(subset=["Sentence"],inplace=True)
meme

305151
919


,Sentence,Noun,Verb,Adjective,Channel,user
6,"Drive to Work #12, #436 and #437 all cover GDS...",[],"[Drive, Work, cover]",[],amateurdesignerhour,U3QMCLQQG
9,Here's one of the articles,[articles],[],[],amateurdesignerhour,U3QMCLQQG
11,This guy failed the test for GDS2 by one quest...,"[guy, test, question, answers]",[failed],[],amateurdesignerhour,U3QMCLQQG
585,Do you have any ideas for themes your stuff wi...,"[ideas, themes, stuff]","[have, explore]",[],aprilfools,U3QMCLQQG
586,I'm very curious about exploring where colors ...,"[colors, talks, color, wheel, time, colors, co...","['m, exploring, go, changing, change, remain, ...","[curious, mechanical, future]",aprilfools,U3L7A68JC
...,...,...,...,...,...,...
233305,I guess it could just be solved by removing al...,"[counters, creatures, memory, issues]","[guess, solved, removing, leaves, eliminates]",[],shittalk,U3FV90TCZ
233306,[Insert 1/1 counter text]\n\nCreatures are 0/0...,"[counter, text, Creatures, leaves, counters, c...",[remove],[],shittalk,U3FV90TCZ
233307,Gives it some added tension/ risk-reward with ...,"[risk, reward, P, T, self, plague, wind]","[Gives, added, doubles, remove]",[],shittalk,U3FV90TCZ
233308,"Do you really think there are memory issues, t...","[memory, issues, version, fun, minigame, encha...","[think, made, figured, be, try, remove, double]","[legendary, entire]",shittalk,UEF135G05


In [176]:
meme[meme['Sentence'].astype(bool)]

,Sentence,Noun,Verb,Adjective,Channel,user
0,<@U3QMCLQQG> has joined the channel,[channel],[joined],[],amateurdesignerhour,U3QMCLQQG
1,"<@U3QMCLQQG> set the channel purpose: Discuss,...","[channel, purpose, study, practice]","[set, prepare]",[],amateurdesignerhour,U3QMCLQQG
3,<@U3QMCLQQG> pinned their File to this channel.,[channel],[pinned],[],amateurdesignerhour,U3QMCLQQG
4,<@U3MTAQBE1> has joined the channel,[channel],[joined],[],amateurdesignerhour,U3MTAQBE1
5,<@U3MTJRAQL> has joined the channel,[channel],[joined],[],amateurdesignerhour,U3MTJRAQL
...,...,...,...,...,...,...
305146,NaN,[https://magic.wizards.com/en/articles/archive...,[],[],youtube-monitor,U3QMCLQQG
305147,This guy failed the test for GDS2 by one quest...,"[guy, test, question, answers]",[failed],[],youtube-monitor,U3QMCLQQG
305148,NaN,[],[],[],youtube-monitor,U3QMCLQQG
305149,<@U3FV90TCZ> has joined the channel,[channel],[joined],[],youtube-monitor,U3FV90TCZ


In [ ]:
wordDataFrame

In [ ]:
specialchara = wordDataFrame.loc[wordDataFrame['Sentence'].str.contains(r'@')==False]
specialchara = specialchara.loc[specialchara['Sentence'].str.contains(r'https')==False]
specialchara = specialchara.loc[specialchara['Sentence'].str.contains(r'::')==False]
specialchara = specialchara.loc[specialchara['Sentence'].str.contains(r'\\')==False]
wordDataFrame = specialchara

In [ ]:
meme = wordDataFrame.drop_duplicates(subset="Sentence",keep=False,inplace=True)
wordDataFrame

In [ ]:
wordDataFrame.loc[wordDataFrame['Noun']=='guy']

In [ ]:
wordDataFrame['Sentence'].value_counts()